In [22]:
import numpy as np
from skimage.morphology import skeletonize, medial_axis, watershed, disk
from skimage import data
from scipy import ndimage as ndi
from scipy.spatial import distance
import matplotlib.pyplot as plt
from skimage.util import invert
from skimage.io import imshow, imread
from skimage.color import rgb2gray
from skimage.feature import peak_local_max, canny
from scipy import ndimage
from skimage.filters import sobel
from skimage.measure import label, regionprops
from skimage.filters import rank

%matplotlib inline
import numpy as np
from skimage import io
from skimage.feature import canny
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.morphology import watershed, disk
from skimage.filters import rank
from skimage.util import img_as_ubyte
from skimage import segmentation
np.set_printoptions(threshold=np.nan)
import math, skimage
from skimage import morphology
from skimage.measure import label, regionprops
from skimage.feature import corner_harris, corner_subpix, corner_peaks

def centroid(image):
    img = image
    #img = imread(image)[:,:,0]
    #img = skimage.transform.rotate(img, 90)
    denoised = rank.median(img, disk(2))
    markers = rank.gradient(denoised, disk(5)) < 10
    markers = ndi.label(markers)[0]
    gradient = rank.gradient(denoised, disk(2))
    labels = watershed(gradient, markers)
    cleared = segmentation.clear_border(labels)
    #imshow(img)
    #print("Preprocessed Image")
    denoised = rank.median(img, disk(2))
    gradient = rank.gradient(denoised, disk(5))
    thres = 30
    gradient[gradient > thres] = 255

    #imshow(gradient)
    labeled_wing = cleared
    regions = regionprops(labeled_wing)
    #print("Number of Regions: ", len(regions))
    #implot = plt.imshow(cleared)

    #Filters regions based on area
    filtered_regions = []
    filtered_regions_area = []
    for props in regions:
        area = props.area
    #     print([500, 600] in props.coords)
        if area>1000 and area<400000:
            filtered_regions.append(props)
            filtered_regions_area.append(area)
    #print("Filtered Regions: ", len(filtered_regions))

    #Plots centroids of regions
    #for region in filtered_regions:
    #    y0, x0 = region.centroid
    #    plt.scatter([x0], [y0])
    #plt.title("Filtered Regions, Plotted Centroids")
    #plt.show()
    #Finds the centroid of all the region centers
    # sum_x, sum_y = 0, 0
    centroids = []
    centroids_x = []
    centroids_y = []
    for region in filtered_regions:
        y0, x0 = region.centroid
        region_centroid = (x0, y0)
        centroids.append(region_centroid)
        centroids_x.append(x0)
        centroids_y.append(y0)
    #     sum_x += x0
    #     sum_y += y0
    # avg_x = sum_x/len(filtered_regions)
    # avg_y = sum_y/len(filtered_regions)
    avg_x = np.mean(centroids_x)
    avg_y = np.mean(centroids_y)

    #print("Center of Regions: ")
    #implot = plt.imshow(labeled_wing)
    #plt.scatter([avg_x], [avg_y], marker="x")
    #plt.title("Center of Regions: ")
    #plt.show()
    
    #Calculates the distance between each centroid to the center
    # implot = plt.imshow(labeled_wing)
    distances_from_center = []
    for centroid in centroids:
        distances_from_center.append(distance.euclidean(centroid, (avg_x, avg_y)))
    # min_dist_centroid = centroids[distances_from_center.index(min(distances_from_center))]
    # plt.scatter([min_dist_centroid[0]], [min_dist_centroid[1]])

    #Normalizes distances from center
    distance_sd = np.std(distances_from_center)
    distance_mean = np.mean(distances_from_center)
    # print("Distance Average", distance_mean, "Distance Standard Dev", distance_sd)
    dist_from_cent_norm = [dist_from_center/distance_mean for dist_from_center in distances_from_center]
#     print("Distance, Scaled: \n", dist_from_cent_norm)

    #Normalizes areas 
    area_sd = np.std(filtered_regions_area)
    area_mean = np.mean(filtered_regions_area)
    area_norm = [area/area_mean for area in filtered_regions_area ]
#     print("Area, Scaled: \n", area_norm)
    
    
    #Weights each region centroid based on distance to the center, and size
    #implot = plt.imshow(labeled_wing)
    distance_weight = 1
    size_weight = 0.3

    weighted_values = []
    for i in range(len(filtered_regions)):
        weighted_values.append(size_weight*area_norm[i] - distance_weight*dist_from_cent_norm[i])

    max_index = weighted_values.index(max(weighted_values))
    center_cell = filtered_regions[max_index]
    # plt.scatter([centroids[max_index][0]], [centroids[max_index][1]])
    center_cell_x, center_cell_y = center_cell.centroid[1], center_cell.centroid[0]
    #plt.scatter([center_cell_x], [center_cell_y])
    return center_cell.centroid[0], center_cell.centroid[1]

# centroid("enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg")
#print("Centroid", centroid("enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg"))

In [31]:
from ipywidgets import interact, fixed

image = imread("enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg")[..., 1]

@interact(deg=(0, 360, 1))
def center_cell_after_rotation_by(deg=0):
    img = skimage.transform.rotate(image, deg, clip = False)
    C = centroid(img)
    
    plt.imshow(img, cmap='gray')
    plt.scatter([C[1]], [C[0]])
    plt.show()

interactive(children=(IntSlider(value=0, description='deg', max=360), Output()), _dom_classes=('widget-interact',))